# NFL Gambling Odds and Results
## We will be manipulating this raw dataset to evaluate the best performing teams against odds provided by Spreadspoke on [Kaggle](https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data). 
### The odds we're derived from [repole.com](http://www.repole.com/sun4cast/data.html) for the seasons ranging from 1978-2013, and for 2013 on betting data reflects lines available at [sportsline.com](sportsline.com) and [aussportsbetting.com](aussportsbetting.com.). 


In [1]:
#importing dependancies
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
#import raw data for discovery and cleaning using pandas
df = pd.read_csv(r'raw_data\spreadspoke_scores.csv')

### Beginning discovery to see what work needs to be done

In [3]:
#display df

df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


In [4]:
# Checking how many rows don't have gambling data or scores, removing those rows

df.isnull().sum()

schedule_date              0
schedule_season            0
schedule_week              0
schedule_playoff           0
team_home                  0
score_home               272
score_away               272
team_away                  0
team_favorite_id        2751
spread_favorite         2751
over_under_line         2761
stadium                    0
stadium_neutral            0
weather_temperature     1548
weather_wind_mph        1564
weather_humidity        5597
weather_detail         10946
dtype: int64

In [5]:
#selecting relevant columns we need 
df = df[['schedule_date','schedule_season', 'schedule_week','team_home','score_home','team_away','score_away',
        'team_favorite_id','spread_favorite','over_under_line']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14073 entries, 0 to 14072
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   schedule_date     14073 non-null  object 
 1   schedule_season   14073 non-null  int64  
 2   schedule_week     14073 non-null  object 
 3   team_home         14073 non-null  object 
 4   score_home        13801 non-null  float64
 5   team_away         14073 non-null  object 
 6   score_away        13801 non-null  float64
 7   team_favorite_id  11322 non-null  object 
 8   spread_favorite   11322 non-null  float64
 9   over_under_line   11312 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 1.1+ MB


In [6]:
# Dropping null values 

df = df.dropna()
df.head()



,schedule_date,schedule_season,schedule_week,team_home,score_home,team_away,score_away,team_favorite_id,spread_favorite,over_under_line
350,1/14/1968,1967,Superbowl,Green Bay Packers,33.0,Oakland Raiders,14.0,GB,-13.5,43
538,1/12/1969,1968,Superbowl,Baltimore Colts,7.0,New York Jets,16.0,IND,-18.0,40
727,1/11/1970,1969,Superbowl,Kansas City Chiefs,23.0,Minnesota Vikings,7.0,MIN,-12.0,39
916,1/17/1971,1970,Superbowl,Baltimore Colts,16.0,Dallas Cowboys,13.0,IND,-2.5,36
1105,1/16/1972,1971,Superbowl,Dallas Cowboys,24.0,Miami Dolphins,3.0,DAL,-6.0,34


In [7]:
#Using datetime function to convert the date columns from objects to date types
df['schedule_date']= pd.to_datetime(df['schedule_date'])
df['schedule_season']= pd.to_datetime(df['schedule_season'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11312 entries, 350 to 13800
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   schedule_date     11312 non-null  datetime64[ns]
 1   schedule_season   11312 non-null  datetime64[ns]
 2   schedule_week     11312 non-null  object        
 3   team_home         11312 non-null  object        
 4   score_home        11312 non-null  float64       
 5   team_away         11312 non-null  object        
 6   score_away        11312 non-null  float64       
 7   team_favorite_id  11312 non-null  object        
 8   spread_favorite   11312 non-null  float64       
 9   over_under_line   11312 non-null  object        
dtypes: datetime64[ns](2), float64(3), object(5)
memory usage: 972.1+ KB


### Adding Necessary Columns 
The following section will add the necessary columns needed to summarize and visualize the NFL teams that have historically been the best and worst at covering spreads and totals. 

In [9]:
#creating total points scored column to compare against over/under line
df['points_scored'] = df['score_home'] + df['score_away']
df

,schedule_date,schedule_season,schedule_week,team_home,score_home,team_away,score_away,team_favorite_id,spread_favorite,over_under_line,points_scored
350,1968-01-14,1970-01-01 00:00:00.000001967,Superbowl,Green Bay Packers,33.0,Oakland Raiders,14.0,GB,-13.5,43,47.0
538,1969-01-12,1970-01-01 00:00:00.000001968,Superbowl,Baltimore Colts,7.0,New York Jets,16.0,IND,-18.0,40,23.0
727,1970-01-11,1970-01-01 00:00:00.000001969,Superbowl,Kansas City Chiefs,23.0,Minnesota Vikings,7.0,MIN,-12.0,39,30.0
916,1971-01-17,1970-01-01 00:00:00.000001970,Superbowl,Baltimore Colts,16.0,Dallas Cowboys,13.0,IND,-2.5,36,29.0
1105,1972-01-16,1970-01-01 00:00:00.000001971,Superbowl,Dallas Cowboys,24.0,Miami Dolphins,3.0,DAL,-6.0,34,27.0
...,...,...,...,...,...,...,...,...,...,...,...
13796,2024-01-21,1970-01-01 00:00:00.000002023,Division,Buffalo Bills,24.0,Kansas City Chiefs,27.0,BUF,-2.5,46,51.0
13797,2024-01-21,1970-01-01 00:00:00.000002023,Division,Detroit Lions,31.0,Tampa Bay Buccaneers,23.0,DET,-6.0,49.5,54.0
13798,2024-01-28,1970-01-01 00:00:00.000002023,Conference,Baltimore Ravens,10.0,Kansas City Chiefs,17.0,BAL,-4.5,44,27.0
13799,2024-01-28,1970-01-01 00:00:00.000002023,Conference,San Francisco 49ers,34.0,Detroit Lions,31.0,SF,-7.5,53.5,65.0


In [10]:
#Removing white space from created column then changing data type to float for further evaluation



df['over_under_line'] = df['over_under_line'].str.strip()

df['over_under_line'] = pd.to_numeric(df['over_under_line'])


In [11]:
#dropping nulls
df = df.dropna()

In [12]:
#changing data types to int for easier calculations later on
df[['over_under_line', 'score_home','score_away']] = df[['over_under_line', 'score_home','score_away']].astype(int)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
#verifying above changes took place
df.dtypes


schedule_date       datetime64[ns]
schedule_season     datetime64[ns]
schedule_week               object
team_home                   object
score_home                   int32
team_away                   object
score_away                   int32
team_favorite_id            object
spread_favorite            float64
over_under_line              int32
points_scored              float64
dtype: object

In [14]:
''' creating over/under_results column to test results of the game that was played vs the point total provided in the data set
In most cases, sports betting jargon will use the terms over, under or push to detail the results of the game vs the total. 
'''
conditions = [
    (df['points_scored'] > df['over_under_line']),
    (df['points_scored'] < df['over_under_line']),
     (df['points_scored'] == df['over_under_line'])
]

results = ['over','under','push']
df['over_under_result'] = np.select(conditions,results)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [15]:
'''Adding column to denote winning team.
this will help us calculate teams with the most wins/ losses etc later on'''

conditions = [
    (df['score_home'] > df['score_away']),
    (df['score_home'] < df['score_away']),
     (df['score_home'] == df['score_away'])
]

results = [df['team_home'],df['team_away'],'Tie']
df['winning_team'] = np.select(conditions,results)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [16]:
''' We need to rename the id's to the winning team to make future df manipulation easier.
First we are listing out all unique ID's, matching them with the unique team names, then using a dictionary to replace 
the names.
'''
df['team_favorite_id'].unique() 



array(['GB', 'IND', 'MIN', 'DAL', 'MIA', 'PIT', 'LVR', 'TB', 'CHI', 'DEN',
       'KC', 'LAR', 'NO', 'NYJ', 'PHI', 'SEA', 'TEN', 'PICK', 'CIN', 'NE',
       'ARI', 'LAC', 'CLE', 'WAS', 'BUF', 'ATL', 'DET', 'NYG', 'SF',
       'BAL', 'JAX', 'CAR', 'HOU', 'LV'], dtype=object)

In [17]:
# getting unique team names 
df['team_home'].unique()


array(['Green Bay Packers', 'Baltimore Colts', 'Kansas City Chiefs',
       'Dallas Cowboys', 'Miami Dolphins', 'Minnesota Vikings',
       'Tampa Bay Buccaneers', 'Buffalo Bills', 'Chicago Bears',
       'Denver Broncos', 'Los Angeles Rams', 'New Orleans Saints',
       'New York Jets', 'Philadelphia Eagles', 'Seattle Seahawks',
       'St. Louis Cardinals', 'Washington Redskins',
       'New England Patriots', 'Detroit Lions', 'New York Giants',
       'Pittsburgh Steelers', 'San Diego Chargers', 'San Francisco 49ers',
       'Atlanta Falcons', 'Cincinnati Bengals', 'Cleveland Browns',
       'Houston Oilers', 'Oakland Raiders', 'Los Angeles Raiders',
       'Indianapolis Colts', 'Phoenix Cardinals', 'Arizona Cardinals',
       'Jacksonville Jaguars', 'St. Louis Rams', 'Carolina Panthers',
       'Baltimore Ravens', 'Tennessee Oilers', 'Tennessee Titans',
       'Houston Texans', 'Los Angeles Chargers',
       'Washington Football Team', 'Las Vegas Raiders',
       'Washington Comman

In [18]:
#dictionary used to swap Id's with names
replacements = {'GB':'Green Bay Packers', 'IND':'Baltimore Colts', 'MIN':'Minnesota Vikings', 'DAL':'Dallas Cowboys', 'MIA':'Miami Dolphins', 
                'PIT':'Pittsburgh Steelers', 'LVR':'Las Vegas Raiders', 'TB':'Tampa Bay Buccaneers', 'CHI':'Chicago Bears', 'DEN':'Denver Broncos',
                 'KC': 'Kansas City Chiefs', 'LAR':'Los Angeles Rams', 'NO': 'New Orleans Saints', 'NYJ': 'New York Jets', 'PHI':'Philadelphia Eagles', 'SEA':'Seattle Seahawks',
                'TEN':'Tennessee Titans', 'PICK':'PICK', 'CIN':'Cincinnati Bengals', 'NE':'New England Patriots',
                'ARI':'Arizona Cardinals', 'LAC': 'Los Angeles Chargers', 'CLE':'Cleveland Browns', 'WAS':'Washington Commanders', 'BUF':'Buffalo Bills',
                'ATL':'Atlanta Falcons', 'DET':'Detroit Lions', 'NYG':'New York Giants', 'SF':'San Francisco 49ers',
                'BAL':'Baltimore Ravens', 'JAX':'Jacksonville Jaguars', 'CAR':'Carolina Panthers', 'HOU':'Houston Texans', 'LV':'Las Vegas Raiders', 'nan':'nan'}

#using replace function to replace names using previously declared dictionary
df['team_favorite_id'] = df['team_favorite_id'].replace(replacements)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [19]:
'''
Swapping old names with new names for consistency, away column then home column below

'''

replacements = {'Oakland Raiders': 'Las Vegas Raiders',
                'Los Angeles Raiders': 'Las Vegas Raiders',
                'San Diego Chargers': 'Los Angeles Chargers',
                'Washington Football Team':'Washington Commanders',
                'Washington Redskins': 'Washington Commanders',
                'Houston Oilers':'Tennessee Titans',
                'St. Luis Rams':'Los Angeles Rams',
                'Phoenix Cardinals':'Arizona Cardinals',
                'Tennessee Oilers':'Tennessee Titans',
                'Baltimore Colts': 'Indianapolis Colts',
                'St. Louis Cardinals':'Arizona Cardinals'
                }

df['team_away'] = df['team_away'].replace(replacements)


c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
'''Swapping old names with new names for consistency'''
df['team_home'] = df['team_home'].replace(replacements)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
# appying to favorite id as well
df['team_favorite_id'] = df['team_favorite_id'].replace(replacements)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
# appying to winning_team as well
df['winning_team'] = df['winning_team'].replace(replacements)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
#verifying changes took place
df.head()

,schedule_date,schedule_season,schedule_week,team_home,score_home,team_away,score_away,team_favorite_id,spread_favorite,over_under_line,points_scored,over_under_result,winning_team
350,1968-01-14,1970-01-01 00:00:00.000001967,Superbowl,Green Bay Packers,33,Las Vegas Raiders,14,Green Bay Packers,-13.5,43,47.0,over,Green Bay Packers
538,1969-01-12,1970-01-01 00:00:00.000001968,Superbowl,Indianapolis Colts,7,New York Jets,16,Indianapolis Colts,-18.0,40,23.0,under,New York Jets
727,1970-01-11,1970-01-01 00:00:00.000001969,Superbowl,Kansas City Chiefs,23,Minnesota Vikings,7,Minnesota Vikings,-12.0,39,30.0,under,Kansas City Chiefs
916,1971-01-17,1970-01-01 00:00:00.000001970,Superbowl,Indianapolis Colts,16,Dallas Cowboys,13,Indianapolis Colts,-2.5,36,29.0,under,Indianapolis Colts
1105,1972-01-16,1970-01-01 00:00:00.000001971,Superbowl,Dallas Cowboys,24,Miami Dolphins,3,Dallas Cowboys,-6.0,34,27.0,under,Dallas Cowboys


In [24]:
'''create column to list underdog teams'''

conditions = [
    (df['team_favorite_id'] == df['team_home']),
    (df['team_favorite_id'] == df['team_away']),
    (df['team_favorite_id'] == 'PICK')

]

results = [df['team_away'],df['team_home'],df['winning_team']]

df['underdog'] = np.select(conditions,results)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [25]:
df.head(30)

,schedule_date,schedule_season,schedule_week,team_home,score_home,team_away,score_away,team_favorite_id,spread_favorite,over_under_line,points_scored,over_under_result,winning_team,underdog
350,1968-01-14,1970-01-01 00:00:00.000001967,Superbowl,Green Bay Packers,33,Las Vegas Raiders,14,Green Bay Packers,-13.5,43,47.0,over,Green Bay Packers,Las Vegas Raiders
538,1969-01-12,1970-01-01 00:00:00.000001968,Superbowl,Indianapolis Colts,7,New York Jets,16,Indianapolis Colts,-18.0,40,23.0,under,New York Jets,New York Jets
727,1970-01-11,1970-01-01 00:00:00.000001969,Superbowl,Kansas City Chiefs,23,Minnesota Vikings,7,Minnesota Vikings,-12.0,39,30.0,under,Kansas City Chiefs,Kansas City Chiefs
916,1971-01-17,1970-01-01 00:00:00.000001970,Superbowl,Indianapolis Colts,16,Dallas Cowboys,13,Indianapolis Colts,-2.5,36,29.0,under,Indianapolis Colts,Dallas Cowboys
1105,1972-01-16,1970-01-01 00:00:00.000001971,Superbowl,Dallas Cowboys,24,Miami Dolphins,3,Dallas Cowboys,-6.0,34,27.0,under,Dallas Cowboys,Miami Dolphins
1294,1973-01-14,1970-01-01 00:00:00.000001972,Superbowl,Miami Dolphins,14,Washington Commanders,7,Miami Dolphins,-1.0,33,21.0,under,Miami Dolphins,Washington Commanders
1483,1974-01-13,1970-01-01 00:00:00.000001973,Superbowl,Miami Dolphins,24,Minnesota Vikings,7,Miami Dolphins,-6.5,33,31.0,under,Miami Dolphins,Minnesota Vikings
1672,1975-01-12,1970-01-01 00:00:00.000001974,Superbowl,Minnesota Vikings,6,Pittsburgh Steelers,16,Pittsburgh Steelers,-3.0,33,22.0,under,Pittsburgh Steelers,Minnesota Vikings
1861,1976-01-18,1970-01-01 00:00:00.000001975,Superbowl,Dallas Cowboys,17,Pittsburgh Steelers,21,Pittsburgh Steelers,-7.0,36,38.0,over,Pittsburgh Steelers,Dallas Cowboys
2064,1977-01-09,1970-01-01 00:00:00.000001976,Superbowl,Minnesota Vikings,14,Las Vegas Raiders,32,Las Vegas Raiders,-4.0,38,46.0,over,Las Vegas Raiders,Minnesota Vikings


In [26]:
#renaming team_favorite id column to favorite for a cleaner look
df = df.rename(columns={'team_favorite_id':'favorite','spread_favorite':'spread'})

In [27]:
#creating point differential column
df['point_differential'] = abs(df['score_home']-df['score_away'])

In [28]:
list(df.columns.values)

['schedule_date',
 'schedule_season',
 'schedule_week',
 'team_home',
 'score_home',
 'team_away',
 'score_away',
 'favorite',
 'spread',
 'over_under_line',
 'points_scored',
 'over_under_result',
 'winning_team',
 'underdog',
 'point_differential']

In [29]:
#reordering dataframe to personal preference 
df = df[['schedule_date','schedule_season','schedule_week',
        'team_home','score_home','team_away','score_away',
        'points_scored','over_under_line','over_under_result','favorite',
        'underdog','point_differential',
        'spread','winning_team']]
df.head()

,schedule_date,schedule_season,schedule_week,team_home,score_home,team_away,score_away,points_scored,over_under_line,over_under_result,favorite,underdog,point_differential,spread,winning_team
350,1968-01-14,1970-01-01 00:00:00.000001967,Superbowl,Green Bay Packers,33,Las Vegas Raiders,14,47.0,43,over,Green Bay Packers,Las Vegas Raiders,19,-13.5,Green Bay Packers
538,1969-01-12,1970-01-01 00:00:00.000001968,Superbowl,Indianapolis Colts,7,New York Jets,16,23.0,40,under,Indianapolis Colts,New York Jets,9,-18.0,New York Jets
727,1970-01-11,1970-01-01 00:00:00.000001969,Superbowl,Kansas City Chiefs,23,Minnesota Vikings,7,30.0,39,under,Minnesota Vikings,Kansas City Chiefs,16,-12.0,Kansas City Chiefs
916,1971-01-17,1970-01-01 00:00:00.000001970,Superbowl,Indianapolis Colts,16,Dallas Cowboys,13,29.0,36,under,Indianapolis Colts,Dallas Cowboys,3,-2.5,Indianapolis Colts
1105,1972-01-16,1970-01-01 00:00:00.000001971,Superbowl,Dallas Cowboys,24,Miami Dolphins,3,27.0,34,under,Dallas Cowboys,Miami Dolphins,21,-6.0,Dallas Cowboys


In [30]:
#creating covering_team column based on list of conditions
conditions = [
    ((df['winning_team']==df['favorite']) & (df['point_differential']> abs(df['spread']))),
    ((df['winning_team'] == df['underdog'])),
    (df['winning_team']==df['favorite']) & (df['point_differential'] < abs(df['spread'])),
    (df['point_differential'] == abs(df['spread'])),
    (df['spread'] 
     == '0.0')
    

]

results = [df['favorite'],df['underdog'],df['underdog'],'PUSH',df['winning_team']]

df['covering_team'] = np.select(conditions,results)

In [31]:
test = df['covering_team'] == 0
test.value_counts()

False    11082
True       168
Name: covering_team, dtype: int64

In [32]:
test = df['winning_team'] == 'Tie'
test.value_counts()

False    11224
True        26
Name: winning_team, dtype: int64

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11250 entries, 350 to 13800
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   schedule_date       11250 non-null  datetime64[ns]
 1   schedule_season     11250 non-null  datetime64[ns]
 2   schedule_week       11250 non-null  object        
 3   team_home           11250 non-null  object        
 4   score_home          11250 non-null  int32         
 5   team_away           11250 non-null  object        
 6   score_away          11250 non-null  int32         
 7   points_scored       11250 non-null  float64       
 8   over_under_line     11250 non-null  int32         
 9   over_under_result   11250 non-null  object        
 10  favorite            11250 non-null  object        
 11  underdog            11250 non-null  object        
 12  point_differential  11250 non-null  int32         
 13  spread              11250 non-null  float64 

In [34]:
#creating boolean mask to create dataframe of only playoff games

'''mask = (df['schedule_week'] == 'Super Bowl') |(df['schedule_week'] == 'Wildcard') |(df['schedule_week'] == 'Division') |(df['schedule_week'] == 'Conference')
playoff_df= df[mask]
playoff_df.head(-25) '''

"mask = (df['schedule_week'] == 'Super Bowl') |(df['schedule_week'] == 'Wildcard') |(df['schedule_week'] == 'Division') |(df['schedule_week'] == 'Conference')\nplayoff_df= df[mask]\nplayoff_df.head(-25) "

### Exporting Dataframe to as a csv file for Tableau Visualizations

In [37]:
from pathlib import Path
filepath = Path('cleaned_data/gambling.csv')
filepath.parent.mkdir(parents=True,exist_ok=True)
df.to_csv(filepath)

